In [1]:
import os 
import sys
import pypdf

from langchain.llms import Ollama
from langchain_chroma import Chroma
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [2]:
#Creating embeddings of the text from the PDF document
def ingest():
    #Load the document
    loader = PyPDFLoader("D:\\Research\PhD@IITM\\My_Research_Papers\\IITM\Published_papers\\Pilla_2022_J_Phys_Condens_Matter.pdf")
    pages = loader.load_and_split()
    
    #Split the pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")

    #Create embeddings
    embedding = FastEmbedEmbeddings()
    #Create Vector store
    Chroma.from_documents(documents=chunks, 
                          embedding=embedding, 
                          persist_directory="./sql_chroma_db")

<>:4: SyntaxWarning: invalid escape sequence '\P'
<>:4: SyntaxWarning: invalid escape sequence '\P'
C:\Users\chemp\AppData\Local\Temp\ipykernel_2280\3473995822.py:4: SyntaxWarning: invalid escape sequence '\P'
  loader = PyPDFLoader("D:\\Research\PhD@IITM\\My_Research_Papers\\IITM\Published_papers\\Pilla_2022_J_Phys_Condens_Matter.pdf")


In [ ]:
#Create an access token from Hugging face and use the same as the read and write token below
from huggingface_hub import login
access_token_read = "hf_nMedcrxscgRruLcCMGPcUHbYyqcGxdcuzasEDy"
access_token_write = "hf_nMedcrxscgRruLcCMGPcUHbYyqcGxdcuzasEDy"
login(token = access_token_read)

In [52]:
#Run this only once to create the vector database 
ingest() # I need to run once only for each new pdf document

Split 15 documents into 50 chunks.


In [53]:
def rag_chain():
    #model definition
    model = Ollama(model="llama3", base_url="http://localhost:11434/", temperature = 0.0)
    #Low temperature = less creativity or variation in the answer by the model

    #prompt definition
    prompt = PromptTemplate.from_template(
        """
        <s> [Instructions] You are a friendly assistant. Answer the question based only on the following context. 
        If you don't know the answer, then reply, No Context availabel for this question {input}. [/Instructions] </s> 
        [Instructions] Question: {input} 
        Context: {context} 
        Answer: [/Instructions]
        """
    )
    
    #Load text embeddings from vector store
    embedding = FastEmbedEmbeddings()
    vector_store = Chroma(persist_directory="./sql_chroma_db", embedding_function=embedding)

    #Create chain
    retriever = vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={
            "k": 3,
            "score_threshold": 0.5,
        },
    )

    document_chain = create_stuff_documents_chain(model, prompt)
    chain = create_retrieval_chain(retriever, document_chain)

    return chain

In [54]:
chain = rag_chain()
print("chain generated")

chain generated


In [55]:
result = chain.invoke({"input": "What are the values of rotational diffusivity used in the paper?"})
print(result["answer"])

The values of rotational diffusivity used in the paper are:

* Dr = 0 (Figure 7(a))
* Dr = 0.01 (Figure 7(b))
* Dr = 1 (Figure 7(c))
* Dr = 4 (Figure 7(d))
* Dr = 10 (Figure 7(e))
* Dr = 100 (Figure 7(f))

Additionally, Figure 8 shows the static structure factor of the system with respect to different rotational diffusivities at constant activity:

* Pe = 0 (Figure 8(a))
* Pe = 4 (Figure 8(b))
* Pe = 15 (Figure 8(c))


In [60]:
result1 = chain.invoke({"input": "Can you give me a list of sections in the paper?"})
print(result1["answer"])

No relevant docs were retrieved using the relevance score threshold 0.5


Answer: 

The paper has the following sections:

1. Introduction
2. Literature Review
3. Methodology
4. Results
5. Discussion
6. Conclusion


In [67]:
#Creating a function for the query search
def ask(query: str):
    #create chain
    chain = rag_chain()
    #invoke chain
    result = chain.invoke({"input": query})
    #print results with source
    print(result["answer"])
    for doc in result["context"]:
        print("Source: ", doc.metadata["source"])

In [70]:
ask("What is active colloid?")

What is active colloid?

Answer: An active colloid is a synthetic colloidal model system that self-propels, mimicking the motion of biological systems. It is engineered using various physical and chemical processes to exhibit non-equilibrium phenomena, such as motility-induced phase separation, re-entrant phase behavior, and rotating crystals.
Source:  D:\Research\PhD@IITM\My_Research_Papers\IITM\Published_papers\Pilla_2022_J_Phys_Condens_Matter.pdf
Source:  D:\Research\PhD@IITM\My_Research_Papers\IITM\Published_papers\Pilla_2022_J_Phys_Condens_Matter.pdf
Source:  D:\Research\PhD@IITM\My_Research_Papers\IITM\Published_papers\Pilla_2022_J_Phys_Condens_Matter.pdf
